In [ ]:
#https://cooltools.readthedocs.io/en/latest/notebooks/viz.html

#Import the packages we will use
#Utilities
import os
import re
import itertools
from itertools import combinations
import glob
import pickle
import argparse

#Data Management
import numpy as np
from numpy import diff
import pandas as pd
import h5py
import scipy
from scipy.stats import linregress
from scipy import ndimage
from functools import partial
from scipy.linalg import toeplitz

#Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
import seaborn as sns
import upsetplot
from upsetplot import UpSet

#Genomics
import pairtools
import cooler
import cooltools
# Import python package for working with cooler files and tools for analysis
import cooler
import cooltools.lib.plotting
import bioframe
from bioframe import overlap
import bbi
from cooltools import insulation

In [ ]:
#Using hg38 aligned files, sampling coolers to same number of interactions between samples/replicates

In [ ]:
outDataDir = '..'

In [ ]:
#conditions
conditions = [
    'WT_Ctrl_R1',
    'WT_ATRA_R1',
    'Top2BKO_Ctrl_R1',
    'Top2BKO_ATRA_R1',
    'WT_Ctrl_R2',
    'WT_ATRA_R2',
    'Top2BKO_Ctrl_R2',
    'Top2BKO_ATRA_R2',
    'WT_Ctrl_R1R2',
    'WT_ATRA_R1R2',
    'Top2BKO_Ctrl_R1R2',
    'Top2BKO_ATRA_R1R2'
]

long_names = {
    'WT_Ctrl_R1' : 'CA-HiC-Dpn-SH-SY5Y-WT-Ctrl-4-51-R1-T1',
    'WT_ATRA_R1' : 'CA-HiC-Dpn-SH-SY5Y-WT-ATRA-5days-4-51-R1-T1',
    'Top2BKO_Ctrl_R1' : 'CA-HiC-Dpn-SH-SY5Y-BKO98-Ctrl-4-51-R1-T1',
    'Top2BKO_ATRA_R1' : 'CA-HiC-Dpn-SH-SY5Y-BKO98-ATRA-5days-4-51-R1-T1',
    'WT_Ctrl_R2' : 'CA-HiC-Dpn-SH-SY5Y-WT-Ctrl-4-52-R2-T1',
    'WT_ATRA_R2' : 'CA-HiC-Dpn-SH-SY5Y-WT-ATRA-5days-4-52-R2-T1',
    'Top2BKO_Ctrl_R2' : 'CA-HiC-Dpn-SH-SY5Y-BKO98-Ctrl-4-52-R2-T1',
    'Top2BKO_ATRA_R2' : 'CA-HiC-Dpn-SH-SY5Y-BKO98-ATRA-5days-4-52-R2-T1',
    'WT_Ctrl_R1R2' : 'CA-HiC-Dpn-SH-SY5Y-WT-Ctrl-4-51-and-4-52-R1R2',
    'WT_ATRA_R1R2' : 'CA-HiC-Dpn-SH-SY5Y-WT-ATRA-5days-4-51-and-4-52-R1R2',
    'Top2BKO_Ctrl_R1R2' : 'CA-HiC-Dpn-SH-SY5Y-BKO98-Ctrl-4-51-and-4-52-R1R2',
    'Top2BKO_ATRA_R1R2' : 'CA-HiC-Dpn-SH-SY5Y-BKO98-ATRA-5days-4-51-and-4-52-R1R2'
}

In [ ]:
#add colors for each sample
sampleColors = {
    'WT_Ctrl_R1' : '#a6cee3',
    'WT_ATRA_R1' : '#1f78b4',
    'Top2BKO_Ctrl_R1' : '#b2df8a',
    'Top2BKO_ATRA_R1' : '#33a02c',
    'WT_Ctrl_R2' : '#a6cee3',
    'WT_ATRA_R2' : '#1f78b4',
    'Top2BKO_Ctrl_R2' : '#b2df8a',
    'Top2BKO_ATRA_R2' : '#33a02c',
    'WT_Ctrl_R1R2' : '#a6cee3',
    'WT_ATRA_R1R2' : '#1f78b4',
    'Top2BKO_Ctrl_R1R2' : '#b2df8a',
    'Top2BKO_ATRA_R1R2' : '#33a02c'
}

sampleLineStyles = {
    'WT_Ctrl_R1' : '--',
    'WT_ATRA_R1' : '--',
    'Top2BKO_Ctrl_R1' : '--',
    'Top2BKO_ATRA_R1' : '--',
    'WT_Ctrl_R2' : ':',
    'WT_ATRA_R2' : ':',
    'Top2BKO_Ctrl_R2' : ':',
    'Top2BKO_ATRA_R2' : ':',
    'WT_Ctrl_R1R2' : '-',
    'WT_ATRA_R1R2' : '-',
    'Top2BKO_Ctrl_R1R2' : '-',
    'Top2BKO_ATRA_R1R2' : '-'
}

samplePlotNames = {
    'WT_Ctrl_R1' : 'WT Ctrl, R1',
    'WT_ATRA_R1' : 'WT ATRA, R1',
    'Top2BKO_Ctrl_R1' : 'BKO Ctrl, R1',
    'Top2BKO_ATRA_R1' : 'BKO ATRA, R1',
    'WT_Ctrl_R2' : 'WT Ctrl, R2',
    'WT_ATRA_R2' : 'WT ATRA, R2',
    'Top2BKO_Ctrl_R2' : 'BKO Ctrl, R2',
    'Top2BKO_ATRA_R2' : 'BKO ATRA, R2',
    'WT_Ctrl_R1R2' : 'WT Ctrl',
    'WT_ATRA_R1R2' : 'WT ATRA',
    'Top2BKO_Ctrl_R1R2' : 'BKO Ctrl',
    'Top2BKO_ATRA_R1R2' : 'BKO ATRA'    
}

In [ ]:
SepConds = [
    'WT_Ctrl_R1',
    'WT_ATRA_R1',
    'Top2BKO_Ctrl_R1',
    'Top2BKO_ATRA_R1',
    'WT_Ctrl_R2',
    'WT_ATRA_R2',
    'Top2BKO_Ctrl_R2',
    'Top2BKO_ATRA_R2',
]

ComboConds = [
    'WT_Ctrl_R1R2',
    'WT_ATRA_R1R2',
    'Top2BKO_Ctrl_R1R2',
    'Top2BKO_ATRA_R1R2'    
]

SepCtrlConds = [
    'WT_Ctrl_R1',
    'WT_Ctrl_R1',
    'WT_ATRA_R1',
    'Top2BKO_Ctrl_R1',
    'WT_Ctrl_R2',
    'WT_Ctrl_R2',
    'WT_ATRA_R2',
    'Top2BKO_Ctrl_R2',
]

SepTreatConds = [
    'WT_ATRA_R1',
    'Top2BKO_Ctrl_R1',
    'Top2BKO_ATRA_R1',
    'Top2BKO_ATRA_R1',
    'WT_ATRA_R2',
    'Top2BKO_Ctrl_R2',
    'Top2BKO_ATRA_R2',
    'Top2BKO_ATRA_R2',   
]

ComboCtrlConds = [
    'WT_Ctrl_R1R2',
    'WT_Ctrl_R1R2',
    'WT_ATRA_R1R2',
    'Top2BKO_Ctrl_R1R2'
]

ComboTreatConds = [
    'WT_ATRA_R1R2',
    'Top2BKO_Ctrl_R1R2',
    'Top2BKO_ATRA_R1R2',
    'Top2BKO_ATRA_R1R2'  
]

In [ ]:
#multi-chromosome heatmaps, 5Mb bins

In [ ]:
#coolers - 5Mb bins
binsize = 5000000

clr_paths_5Mb = {}
for cond in conditions:
    clr_paths_5Mb[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
    
clrs5Mb = {
    cond: cooler.Cooler(clr_paths_5Mb[cond]) for cond in conditions
}

In [ ]:
import cooltools.lib.plotting

In [ ]:
ComboConds

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")

for cond in ComboConds:
    
    binsize = 5000000

    opts = dict(
        vmin=-3.5, #change scale range here
        vmax=-1,
        cmap='fall'
    )

    fig = plt.figure(figsize=(2.5, 2))
    gs00 = GridSpec(nrows = 1, ncols = 2, width_ratios = [20] + [1], wspace=0.1)
    gs_cb = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.1, subplot_spec = gs00[1])

    #how many heatmaps
    gs0 = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.3, subplot_spec = gs00[0])
    gs1 = GridSpecFromSubplotSpec(nrows = 1, ncols = 1, wspace=0.1, hspace = 0.05, subplot_spec = gs0[0])

    c = clrs5Mb[cond]
    cis = c.matrix(balance = True)[0:, 0:]
    
    n,j = np.indices(cis.shape)
    n = n.flatten()
    j = j.flatten()
    # fill NaNs inside of the desired selection:
    nan_band_selector = (np.abs(n-j) < 2) #fills 2 bins on diagonal with nan
    cis[n[nan_band_selector],j[nan_band_selector]] = np.nan
    
    cisbins = c.bins()[0:]
    
    #plot heatmap
    ax = plt.subplot(gs1[0, 0])
    img = ax.matshow( 
        np.log10(cis),
        **opts)
    ax.axes.xaxis.set_visible(False)   
    ax.axes.yaxis.set_visible(False)

    ax.set_aspect('equal')
    plt.title(samplePlotNames[cond] + '\n')
    
    #color bar in it's own axis, and own gridspace
    colorAx = plt.subplot(gs_cb[0, 0])
    cb = plt.colorbar(img, cax = colorAx, extend = 'both')
    cb.set_label('log10(interactions)')
    #save separately for each condition
    plt.savefig(f"{outDataDir}/figures/{cond}_R1R2_AllbyAllHeatmaps_500MbBins_MinusDiag.png", dpi = 300, bbox_inches = 'tight')

In [ ]:
#Difference heatmap

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")

#ratios for main figure - R1 + R2 combined only, 5Mb bins
binsize = 5000000

for DMSOcond, Treatcond in zip(ComboCtrlConds, ComboTreatConds):
    
    opts = dict(
       vmin=-0.4,
       vmax=0.4,
       cmap='coolwarm'
    )

    fig = plt.figure(figsize=(2.5, 2))

    gs00 = GridSpec(nrows = 1, ncols = 2, width_ratios = [20] + [1], wspace=0.1)

    gs_cb = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.1, subplot_spec = gs00[1])

    #how many heatmaps
    gs0 = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.3, subplot_spec = gs00[0])
    
    treatc = clrs5Mb[Treatcond]
    treatcis = treatc.matrix(balance = True)[0:, 0:]
    treatcisbins = treatc.bins()[0:]
    
    treatLog10 = np.log10(treatcis)
    
    dmsoc = clrs5Mb[DMSOcond]
    dmsocis = dmsoc.matrix(balance = True)[0:, 0:]
    dmsocisbins = dmsoc.bins()[0:]
    
    dmsoLog10 = np.log10(dmsocis)
    
    ratiocis = treatLog10-dmsoLog10
    k,j = np.indices(ratiocis.shape)
    k = k.flatten()
    j = j.flatten()
    # fill NaNs inside of the desired selection:
    nan_band_selector = (np.abs(k-j) < 2) #fills 2 bins on diagonal with nan
    ratiocis[k[nan_band_selector],j[nan_band_selector]] = np.nan
    
    ax = plt.subplot(gs0[0])
    img = ax.matshow(
        ratiocis, 
        **opts)
    ax.axes.xaxis.set_visible(False)   
    ax.axes.yaxis.set_visible(False)
    plt.title(samplePlotNames[Treatcond] + '/\n' + samplePlotNames[DMSOcond])
    ax.set_aspect('equal')
    
    # color bar in it's own axis, and own gridspace
    colorAx = plt.subplot(gs00[1])
    cb = plt.colorbar(img, cax = colorAx, extend = 'both')
    cb.set_label('log10(Treated/Ctrl)')

    plt.savefig(f"{outDataDir}/figures/{Treatcond}vs{DMSOcond}_R1R2_AllbyAll_minusdiag_Ratio_500KbBins.png", dpi = 300, bbox_inches = 'tight')

In [ ]:
#chrom 1-5 only

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")

binsize = 5000000

for cond in ComboConds:
    
    opts = dict(
        vmin=-3.5, #change scale range here
        vmax=-1,
        cmap='fall'
    )

    fig = plt.figure(figsize=(2.5, 2))

    gs00 = GridSpec(nrows = 1, ncols = 2, width_ratios = [20] + [1], wspace=0.1)

    gs_cb = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.1, subplot_spec = gs00[1])

    #how many heatmaps
    gs0 = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.3, subplot_spec = gs00[0])

    gs1 = GridSpecFromSubplotSpec(nrows = 1, ncols = 1, wspace=0.1, hspace = 0.05, subplot_spec = gs0[0])

    c = clrs5Mb[cond]
    cis = c.matrix(balance = True)[0:214, 0:214]
    
    n,j = np.indices(cis.shape)
    n = n.flatten()
    j = j.flatten()
    # fill NaNs inside of the desired selection:
    nan_band_selector = (np.abs(n-j) < 2) #fills 2 bins on diagonal with nan
    cis[n[nan_band_selector],j[nan_band_selector]] = np.nan
    
    cisbins = c.bins()[0:214]
    
    #plot heatmap
    ax = plt.subplot(gs1[0, 0])
    img = ax.matshow( #can change this to plt.imshow instead probably?
        np.log10(cis),
        **opts)
    ax.axes.xaxis.set_visible(False)   
    ax.axes.yaxis.set_visible(False)

    ax.set_aspect('equal')
    plt.title(samplePlotNames[cond] + '\n')
    
# color bar in it's own axis, and own gridspace
    colorAx = plt.subplot(gs_cb[0, 0])
    cb = plt.colorbar(img, cax = colorAx, extend = 'both')
    cb.set_label('log10(interactions)')

    #plt.suptitle(f'Chr14 Heatmaps and Eigen1 200kb Bins')
    plt.savefig(f"{outDataDir}/figures/{cond}_R1R2_chr1to5_heatmaps_500MbBins_MinusDiag.png", dpi = 300, bbox_inches = 'tight')

In [ ]:
#Difference heatmap

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")

#ratios for main figure - R1 + R2 combined only, 5Mb bins
binsize = 5000000

for DMSOcond, Treatcond in zip(ComboCtrlConds, ComboTreatConds):
    
    opts = dict(
       vmin=-0.4,
       vmax=0.4,
        cmap='coolwarm'
    )

    fig = plt.figure(figsize=(2.5, 2))

    gs00 = GridSpec(nrows = 1, ncols = 2, width_ratios = [20] + [1], wspace=0.1)

    gs_cb = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.1, subplot_spec = gs00[1])

    #how many heatmaps
    gs0 = GridSpecFromSubplotSpec(nrows=1, ncols=1, wspace = 0.3, subplot_spec = gs00[0])

    
    treatc = clrs5Mb[Treatcond]
    treatcis = treatc.matrix(balance = True)[0:214, 0:214]
    treatcisbins = treatc.bins()[0:214]
    
    treatLog10 = np.log10(treatcis)
    
    dmsoc = clrs5Mb[DMSOcond]
    dmsocis = dmsoc.matrix(balance = True)[0:214, 0:214]
    dmsocisbins = dmsoc.bins()[0:214]
    
    dmsoLog10 = np.log10(dmsocis)
    
    ratiocis = treatLog10-dmsoLog10
    k,j = np.indices(ratiocis.shape)
    k = k.flatten()
    j = j.flatten()
    # fill NaNs inside of the desired selection:
    nan_band_selector = (np.abs(k-j) < 2) #fills 2 bins on diagonal with nan
    ratiocis[k[nan_band_selector],j[nan_band_selector]] = np.nan
    
    ax = plt.subplot(gs0[0])
    img = ax.matshow(
        ratiocis, 
        **opts)
    ax.axes.xaxis.set_visible(False)   
    ax.axes.yaxis.set_visible(False)
    plt.title(samplePlotNames[Treatcond] + '/\n' + samplePlotNames[DMSOcond])
    ax.set_aspect('equal')
    
    # color bar in it's own axis, and own gridspace
    colorAx = plt.subplot(gs00[1])
    cb = plt.colorbar(img, cax = colorAx, extend = 'both')
    cb.set_label('log10(Treated/Ctrl)')

    plt.savefig(f"{outDataDir}/figures/{Treatcond}vs{DMSOcond}_R1R2_chr1to5_minusdiag_Ratio_500KbBins.png", dpi = 300, bbox_inches = 'tight')

In [ ]:
#Heatmaps with eigenvectors - 250kb bins

In [ ]:
#coolers - 250kb bins
binsize = 250000

clr_paths_250kb = {}
for cond in conditions:
    clr_paths_250kb[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
    
clrs250kb = {
    cond: cooler.Cooler(clr_paths_250kb[cond]) for cond in conditions
}

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
# create a view with chromosome arms using chromosome sizes and definition of centromeres
hg38_arms = bioframe.make_chromarms(hg38_chromsizes,  hg38_cens)
hg38_fullchroms = bioframe.make_viewframe(hg38_chromsizes)

In [ ]:
good_chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13',
              'chr14', 'chr16', 'chr18', 'chr19', 'chr20', 'chr21']

In [ ]:
# Select only chromosomes that are present in the good chromosomes
hg38_chromsizes = hg38_chromsizes.loc[good_chroms]
hg38_arms = hg38_arms[hg38_arms.chrom.isin(good_chroms)].reset_index(drop=True)
hg38_fullchroms = hg38_fullchroms[hg38_fullchroms.chrom.isin(good_chroms)].reset_index(drop = True)

In [ ]:
bins = cooler.binnify(hg38_chromsizes, binsize)

In [ ]:
from bioframe.io.resources import UCSCClient
mrna = UCSCClient('hg38').fetch_mrna()

In [ ]:
genecov = bioframe.frac_gene_coverage(bins, mrna) 

In [ ]:
#Get compartments, flip based on gene density, all chromosomes
#Not sorting by variance compared to gene density, instead by variance explained of data

lam = {}
eigs = {}

for cond in conditions:
    lam[cond], eigs[cond] = cooltools.eigs_cis(
        clrs250kb[cond], 
        genecov,
        n_eigs=3, 
        ignore_diags=2,
        view_df = hg38_fullchroms
    )

In [ ]:
for cond in conditions:
    # Save text files
    lam[cond].to_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.byarm.eigs.cis.lam.txt', sep='\t')

In [ ]:
#plot eig1 for each region to make sure they were detecting compartments and not something else...
fig = plt.figure(figsize=(15, 2 * len(hg38_fullchroms)))

gs1 = GridSpec(nrows = len(hg38_fullchroms), ncols = 1, hspace = 0.8)

for i, chrom in enumerate(hg38_fullchroms.itertuples()):
    ax = plt.subplot(gs1[i])
    for cond in conditions:
        loc_eig = bioframe.select(eigs[cond], chrom[1:4])
        ax.plot(
            loc_eig['start'],
            loc_eig['E1'],
            label = cond,
            color = sampleColors[cond],
            ls = sampleLineStyles[cond]
        )
        plt.axhline(0,ls='--',lw=0.5,color='gray')
        plt.ylabel('E1')
        plt.xlabel('position, bp')
        plt.title(chrom)
        plt.legend()
        plt.ylim(-2, 2)

plt.savefig(f'{outDataDir}/figures/Eig1_250kb_bychrom_mapq30.png', dpi = 300)

In [ ]:
#Looks good except for chr18 and chr21 - need to flip Top2B KO for chr18 (all) and WT Ctrl R1 + R2, need to flip Top2BKO ATRA R1 and R1R2 for chr21.

for cond in ['WT_ATRA_R1', 'WT_ATRA_R2', 'WT_ATRA_R1R2', ]:
    eigs[cond].loc[eigs[cond]['chrom'] == 'chr18', 'E1'] = (eigs[cond].loc[eigs[cond]['chrom'] == 'chr18', 'E1'])*-1
    
for cond in ['Top2BKO_ATRA_R1', 'Top2BKO_ATRA_R1R2']:
    eigs[cond].loc[eigs[cond]['chrom'] == 'chr21', 'E1'] = (eigs[cond].loc[eigs[cond]['chrom'] == 'chr21', 'E1'])*-1

In [ ]:
#plot eig1 for each region to make sure they were detecting compartments and not something else...
fig = plt.figure(figsize=(15, 2 * len(hg38_fullchroms)))

gs1 = GridSpec(nrows = len(hg38_fullchroms), ncols = 1, hspace = 0.8)

for i, chrom in enumerate(hg38_fullchroms.itertuples()):
    ax = plt.subplot(gs1[i])
    for cond in conditions:
        loc_eig = bioframe.select(eigs[cond], chrom[1:4])
        ax.plot(
            loc_eig['start'],
            loc_eig['E1'],
            label = cond,
            color = sampleColors[cond],
            ls = sampleLineStyles[cond]
        )
        plt.axhline(0,ls='--',lw=0.5,color='gray')
        plt.ylabel('E1')
        plt.xlabel('position, bp')
        plt.title(chrom)
        plt.legend()
        plt.ylim(-2, 2)

plt.savefig(f'{outDataDir}/figures/Corrected_Eig1_250kb_bychrom_mapq30.png', dpi = 300)

In [ ]:
for cond in conditions:
    # Save text files
    eigs[cond].to_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.byarm.eigs.cis.vecs.txt', sep='\t', index=False, na_rep = 'nan')
    
    # Save bedGraph track
    eigs[cond][['chrom', 'start', 'end', 'E1']].to_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.byarm.eigs.cis.vecs.E1.bedGraph', sep='\t', index=False, na_rep = 'nan', header = False)
    # Save bigwig track
    bioframe.to_bigwig(eigs[cond], hg38_chromsizes, f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.byarm.eigs.cis.vecs.E1.bw', 'E1')


In [ ]:
#read in eigs
eigs = {}
for cond in conditions:
    eigs[cond] = pd.read_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.byarm.eigs.cis.vecs.txt', sep = '\t')

In [ ]:
import cooltools.lib.plotting

In [ ]:
sns.set_context("paper")
sns.set_style('ticks')

for cond in ComboConds:
    
    plottingregion = 'chr14:17086761-107043718'
    chromarm = ('chr14', 17086761, 107043718)

    opts = dict(
        vmin=-4, #change scale range here
        vmax=-2,
        cmap='fall'
    )

    fig = plt.figure(figsize=(2.5, 2.5))

    gs00 = GridSpec(nrows = 1, ncols = 2, width_ratios = [20] + [1], wspace=0.5)

    gs_cb = GridSpecFromSubplotSpec(nrows=2, ncols=1, wspace = 0.1, subplot_spec = gs00[1], hspace = 0.05, 
                                    height_ratios = [6, 1])

    #how many heatmaps
    gs0 = GridSpecFromSubplotSpec(nrows=1, ncols=1, subplot_spec = gs00[0])
    
    gs1 = GridSpecFromSubplotSpec(nrows = 2, ncols = 1, height_ratios=[6, 1], wspace=0.1, 
                                  hspace = 0.05, subplot_spec = gs0[0])

    c = clrs250kb[cond]
    cis = c.matrix(balance = True).fetch(plottingregion)
    
    n,j = np.indices(cis.shape)
    n = n.flatten()
    j = j.flatten()
    # fill NaNs inside of the desired selection:
    nan_band_selector = (np.abs(n-j) < 2) #fills 2 bins on diagonal with nan
    cis[n[nan_band_selector],j[nan_band_selector]] = np.nan
    
    cisbins = c.bins().fetch(plottingregion)
    mids = cisbins['end'] - binsize/2
    mids = mids.reset_index(drop=True)
    
    #plot heatmap
    ax = plt.subplot(gs1[0, 0])
    img = ax.matshow( #can change this to plt.imshow instead probably?
        np.log10(cis), 
        extent=[
            mids[0]/1000000, 
            mids.iloc[-1]/1000000, 
            mids.iloc[-1]/1000000, 
            mids[0]/1000000], 
        **opts)
    ax.xaxis.set_visible(False)
    ax.yaxis.tick_right()    
    ax.set_aspect('equal')
    plt.title(samplePlotNames[cond] + '\n')
    
    # barplot of eigenvalues
    ax1 = plt.subplot(gs1[1, 0])
    plt.ylim(-2, 2) 
    plt.xlim(chromarm[1]/1000000, chromarm[2]/1000000)
    img2 = ax1.bar(
        x = list((eigs[cond][eigs[cond].chrom == chromarm[0]]['start'] + binsize/2)/1000000), 
        height = list(eigs[cond][eigs[cond].chrom == chromarm[0]]['E1']),
        width = 0.25,
        color=eigs[cond].E1[eigs[cond].chrom == chromarm[0]].apply(lambda x: 'r' if x>0 else 'b'),
        alpha = 1,
        linewidth = 0
                  )
    plt.ylabel('Eig 1')
    plt.xlabel('Mb')
    ax1.xaxis.set_ticks(np.arange(20, 110, 10)) #This specifically sets ticks to this range, to match y axis
    fig.add_subplot(ax1)
    
    # color bar in it's own axis, and own gridspace
    colorAx = plt.subplot(gs_cb[0, 0])
    cb = plt.colorbar(img, cax = colorAx, extend = 'both')
    cb.set_label('log10(interactions)')

    plt.savefig(f"{outDataDir}/figures/{cond}_R1R2Combined_FullChr14Heatmaps_250kbBins_MinusDiag_WithEigen1_Barplot_columns.png", dpi = 300, bbox_inches = 'tight')

In [ ]:
#Difference heatmap

In [ ]:
#ratios for main figure - R1 + R2 combined only, 250kb bins

for DMSOcond, Treatcond in zip(ComboCtrlConds, ComboTreatConds):
    
    plottingregion = 'chr14:17086761-107043718'
    chromarm = ('chr14', 17086761, 107043718)

    opts = dict(
        vmin=-.5,
        vmax=.5,
        cmap='coolwarm'
    )

    fig = plt.figure(figsize=(2.5, 2.5))

    gs00 = GridSpec(nrows = 1, ncols = 2, width_ratios = [20] + [1], wspace=0.5)

    gs_cb = GridSpecFromSubplotSpec(nrows=2, ncols=1, wspace = 0.1, subplot_spec = gs00[1], hspace = 0.05, 
                                    height_ratios = [6, 1])

    #how many heatmaps
    gs0 = GridSpecFromSubplotSpec(nrows=1, ncols=1, subplot_spec = gs00[0])
    gs1 = GridSpecFromSubplotSpec(nrows = 2, ncols = 1, height_ratios=[6, 1], wspace=0.1, 
                                  hspace = 0.05, subplot_spec = gs0[0])
    
    treatc = clrs250kb[Treatcond]
    treatcis = treatc.matrix(balance = True).fetch(plottingregion)
    treatcisbins = treatc.bins().fetch(plottingregion)
    treatmids = treatcisbins['end'] - binsize/2
    treatmids = treatmids.reset_index(drop=True)
    treatLog10 = np.log10(treatcis)
    
    dmsoc = clrs250kb[DMSOcond]
    dmsocis = dmsoc.matrix(balance = True).fetch(plottingregion)
    dmsocisbins = dmsoc.bins().fetch(plottingregion)
    dmsomids = dmsocisbins['end'] - binsize/2
    dmsomids = dmsomids.reset_index(drop=True)
    dmsoLog10 = np.log10(dmsocis)
    
    ratiocis = treatLog10-dmsoLog10
    k,j = np.indices(ratiocis.shape)
    k = k.flatten()
    j = j.flatten()
    # fill NaNs inside of the desired selection:
    nan_band_selector = (np.abs(k-j) < 2) #fills 2 bins on diagonal with nan
    ratiocis[k[nan_band_selector],j[nan_band_selector]] = np.nan
    
    ax = plt.subplot(gs1[0, 0])
    img = ax.matshow(
        ratiocis, extent=[treatmids[0]/1000000, treatmids.iloc[-1]/1000000, treatmids.iloc[-1]/1000000, treatmids[0]/1000000], 
        **opts)
    ax.axes.xaxis.set_visible(False)   
    ax.yaxis.tick_right() 
    plt.title(samplePlotNames[Treatcond] + '/\n' + samplePlotNames[DMSOcond])
    ax.set_aspect('equal')
    
    # barplot of eigenvalues - control and treatment - lineplots
    ax1 = plt.subplot(gs1[1, 0])
    plt.ylim(-2, 2) 
    plt.xlim(chromarm[1]/1000000, chromarm[2]/1000000)
    ax1.plot(
        list((eigs[DMSOcond][eigs[DMSOcond].chrom == chromarm[0]]['start'] + binsize/2)/1000000), 
        list(eigs[DMSOcond][eigs[DMSOcond].chrom == chromarm[0]]['E1']), color = sampleColors[DMSOcond], 
        label = samplePlotNames[DMSOcond])
    ax1.plot(
        list((eigs[Treatcond][eigs[Treatcond].chrom == chromarm[0]]['start'] + binsize/2)/1000000), 
        list(eigs[Treatcond][eigs[Treatcond].chrom == chromarm[0]]['E1']), color = sampleColors[Treatcond], 
        label = samplePlotNames[Treatcond])
    plt.ylabel('Eig 1')
    plt.xlabel('Mb')
    ax1.xaxis.set_ticks(np.arange(20, 110, 10)) #This specifically sets ticks to this range, to match y axis
    ax1.legend(bbox_to_anchor=(1.04,1), frameon = False)
    fig.add_subplot(ax1)
    
    # color bar in it's own axis, and own gridspace
    colorAx = plt.subplot(gs_cb[0])
    cb = plt.colorbar(img, cax = colorAx, extend = 'both')
    cb.set_label('log10(Treated/Ctrl)')

    plt.savefig(f"{outDataDir}/figures/{Treatcond}vs{DMSOcond}_R1R2Combined_Chr14DifferenceHeatmaps_minusdiag_250KbBins_LargerColorScale.png".format(outDataDir), dpi = 300, bbox_inches = 'tight')

In [ ]:
#Saddleplots and strength with OWN eig1

In [ ]:
#coolers - 250kb bins
binsize = 250000

clr_paths_250kb = {}
for cond in conditions:
    clr_paths_250kb[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
    
clrs250kb = {
    cond: cooler.Cooler(clr_paths_250kb[cond]) for cond in conditions
}

In [ ]:
#need to filter out arms without any data - from gfudenberg (https://github.com/open2c/cooltools/issues/330)
bins = clrs250kb['WT_Ctrl_R1'].bins()[:].copy()
bins['coverage'] = cooltools.coverage(clrs250kb['WT_Ctrl_R1'])[1]
bins

In [ ]:
view_coverage = bioframe.overlap(hg38_arms, bins).groupby('name').sum('coverage_')
view_coverage

In [ ]:
view_coverage['coverage_'].values > 1e4

In [ ]:
hg38_arms_filtered = bioframe.make_viewframe(hg38_arms[hg38_arms['name'].isin(view_coverage[view_coverage['coverage_'].values > 1e4].index)])

In [ ]:
hg38_arms_filtered

In [ ]:
hg38_arms_filtered.to_csv(f'{outDataDir}/data/hg38_arms_filtered.bed', index = False, header = None, sep = '\t')

In [ ]:
for cond in conditions:
    in_fname = clr_paths_250kb[cond]
    region_fname = f'{outDataDir}/data/hg38_arms_filtered.bed'
    out_fname = f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.expected.cis.cli.tsv'
    !bsub -q short -W 01:00 -e ./lsf_jobs/LSB_%J.err -o ./lsf_jobs/LSB_%J.log \
        -n 4 -R span[hosts=1] -R select[ib] -R rusage[mem=4000] -R select[rh=8] -N \
        "cooltools expected-cis -p 4 -o $out_fname --clr-weight-name weight --ignore-diags 2 $in_fname --regions $region_fname"


In [ ]:
#Saddle strength - AA, BB, AA + BB

In [ ]:
#AA vs BB compartment scores
def saddle_strengthAB(S, C):
    """
    Parameters
    ----------
    S, C : 2D arrays, square, same shape
        Saddle sums and counts, respectively
        
    Returns
    -------
    2x1D array
    Ratios of cumulative corner interaction scores, where the saddle data is 
    separately AA/AB and BB/BA corners with increasing extent
    
    """
    m, n = S.shape
    if m != n:
        raise ValueError("`saddledata` should be square.")

    ratiosA = np.zeros(n)
    for k in range(1, n):
        intra_sumA = S[n-k:n, n-k:n].sum() 
        intra_countA = C[n-k:n, n-k:n].sum()
        intraA = intra_sumA / intra_countA
        
        inter_sum = S[0:k, n-k:n].sum() + S[n-k:n, 0:k].sum()
        inter_count =  C[0:k, n-k:n].sum() + C[n-k:n, 0:k].sum()
        inter = inter_sum / inter_count
        
        ratiosA[k] = intraA / inter
        
    ratiosB = np.zeros(n)
    for k in range(1, n):
        intra_sumB = S[0:k, 0:k].sum()
        intra_countB = C[0:k, 0:k].sum()
        intraB = intra_sumB / intra_countB
        
        inter_sum = S[0:k, n-k:n].sum() + S[n-k:n, 0:k].sum()
        inter_count =  C[0:k, n-k:n].sum() + C[n-k:n, 0:k].sum()
        inter = inter_sum / inter_count
        
        ratiosB[k] = intraB / inter
    
    ratios = {
        'A' : ratiosA,
        'B' : ratiosB
    }
    
    return ratios

In [ ]:
# use this if expected already run, will be much faster
cis_exp = {}

for cond in conditions:
    cis_exp[cond] = pd.read_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.expected.cis.cli.tsv', sep='\t')

In [ ]:
cis_exp[cond] 

In [ ]:
eigs = {}
for cond in conditions:
    eigs[cond] = pd.read_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.byarm.eigs.cis.vecs.txt', sep = '\t')


In [ ]:
eigs[cond]

In [ ]:
#Saddle strength - all distances

In [ ]:
#Use the function to return saddledata (just good chromosomes, by arm)

sums_ownE1 = {}
counts_ownE1 = {}

for cond in conditions:
    sums_ownE1[cond], counts_ownE1[cond] = cooltools.api.saddle.saddle(
            clr = clrs250kb[cond], 
            expected = cis_exp[cond], 
            contact_type = 'cis',
            view_df = hg38_arms_filtered.reset_index(drop = True),
            track = eigs[cond][['chrom', 'start', 'end', 'E1']],
            qrange = (0.02, 0.98),
            n_bins = 50,
            verbose = False
        )

In [ ]:
#Compartment strength - AA vs BB

In [ ]:
#Own Eig1

In [ ]:
#AA vs BB strength
strengthAB_ownE1 = {}
for cond in conditions:
    strengthAB_ownE1[cond] = saddle_strengthAB(sums_ownE1[cond], counts_ownE1[cond])

In [ ]:
SepTreatConds

In [ ]:
SepCtrlConds

In [ ]:
#log2 ratio of treatment/control saddle strength within each replicate
sep_log2_treatvsctrl = {}
for comp in ['A', 'B']:
    sep_log2_treatvsctrl[comp] = {}
    for treat, ctrl in zip(SepTreatConds, SepCtrlConds):
        sep_log2_treatvsctrl[comp][f'{treat}vs{ctrl}'] = np.log2(strengthAB_ownE1[treat][comp]) - np.log2(strengthAB_ownE1[ctrl][comp])
        

In [ ]:
#Plot bargraph of avg and dots of replicates log2(treat/ctrl) for AA and BB comp strength
#10 bin square - top 20% A or B

comp_score_df = pd.DataFrame(columns = ['Comparison', 'Compartment', 'Replicate', 'Label', 'Score'])

repdict = {
    'WT_Ctrl_R1' : 'R1',
    'Top2BKO_Ctrl_R1' : 'R1',
    'WT_ATRA_R1' : 'R1',
    'Top2BKO_ATRA_R1' : 'R1',    
    'WT_Ctrl_R2' : 'R2',
    'Top2BKO_Ctrl_R2' : 'R2',
    'WT_ATRA_R2' : 'R2',
    'Top2BKO_ATRA_R2' : 'R2',
}

labeldict = {
    'WT_Ctrl_R1' : 'WT Ctrl',
    'Top2BKO_Ctrl_R1' : 'BKO Ctrl',
    'WT_ATRA_R1' : 'WT ATRA',
    'Top2BKO_ATRA_R1' : 'BKO ATRA',
    'WT_Ctrl_R2' : 'WT Ctrl',
    'Top2BKO_Ctrl_R2' : 'BKO Ctrl',
    'WT_ATRA_R2' : 'WT ATRA',
    'Top2BKO_ATRA_R2' : 'BKO ATRA',
}

labelPlotColors = {
    'WT Ctrl' : '#17BECF',
    'BKO Ctrl' : '#D62728',
    'WT ATRA' : '#574D68',
    'BKO ATRA' : '#C6A15B',
}

In [ ]:
#Plot actual comp strength (not vs control), cis

In [ ]:
comp_score_df_cis = pd.DataFrame(columns = ['Condition', 'Compartment', 'Replicate', 'Label', 'Score'])

for cond in conditions[0:8]:
    for comp in ['A', 'B']:
            comp_score_df_cis = comp_score_df_cis.append({
                'Condition' : cond, 
                'Compartment' : comp, 
                'Replicate' : repdict[cond], 
                'Label' : labeldict[cond],
                'Score' : strengthAB_ownE1[cond][comp][9]
            }, ignore_index = True)


In [ ]:
#https://stackoverflow.com/questions/64223870/seaborn-overlap-swarmplot-on-barplot
sns.set_style("ticks")
sns.set_context("paper")
cmap_bar = sns.color_palette(['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c'])
gs = GridSpec(nrows= 1, ncols=1, wspace = 0.6, hspace = 0.6)

plt.figure(figsize=(2, 2))

ax = plt.subplot(gs[0])

sns.stripplot(x='Compartment', 
                  y='Score', 
                  hue='Label', 
                  dodge=True, 
                  data=comp_score_df_cis, 
                  jitter = False, 
                  palette = cmap_bar, 
                  ax = ax)
ax1 = sns.barplot(x='Compartment', 
                      y='Score', 
                      hue='Label', 
                      data=comp_score_df_cis, 
                      palette = cmap_bar, 
                      alpha = 0.5, 
                      ci = False, 
                      ax = ax)
    
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[4:8], labels[4:8], bbox_to_anchor=(1.04,1), frameon = False)
 
plt.title(f'Intrachromosomal\nCompartment Strength')
plt.ylabel('Saddle Strength')  
plt.ylim(0, 10)
plt.xlabel('Compartment')
        
plt.savefig(f'{outDataDir}/figures/R1R2_HiC_compstrength_notvsctrl_Eig1.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Compartment strength analysis - use own eigens, calculate separately for reach replicate, trans saddle

In [ ]:
# create pairwise combinations of chromosomes for calculating average interactions:
regions = bioframe.core.construction.add_ucsc_name_column(bioframe.from_any(hg38_chromsizes))
region_pairs = list(combinations(regions[['chrom', 'start', 'end']].values, 2))

In [ ]:
region_pairs

In [ ]:
regions.to_csv(f'{outDataDir}/data/hg38_chroms.bed', index = False, sep = '\t', header = None)

In [ ]:
for cond in conditions:
    in_fname = clr_paths_250kb[cond]
    region_fname = f'{outDataDir}/data/hg38_chroms.bed'
    out_fname = f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.expected.trans.cli.tsv'
    !bsub -q short -W 01:00 -e ./lsf_jobs/LSB_%J.err -o ./lsf_jobs/LSB_%J.log \
        -n 4 -R span[hosts=1] -R select[ib] -R rusage[mem=4000] -R select[rh=8] -N \
        "cooltools expected-trans -p 4 -o $out_fname $in_fname --view $region_fname"


In [ ]:
#Saddle strength - AA, BB, AA + BB

In [ ]:
# use this if expected already run, will be much faster
trans_exp = {}

for cond in conditions:
    trans_exp[cond] = pd.read_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.expected.trans.cli.tsv', sep='\t')

In [ ]:
trans_exp[cond]

In [ ]:
regions

In [ ]:
#Use the function to return saddledata for each band size (just good chromosomes, by arm)

trans_sums = {}
trans_counts = {}

for cond in conditions:
    trans_sums[cond], trans_counts[cond] = cooltools.api.saddle.saddle(
        clr = clrs250kb[cond], 
        expected = trans_exp[cond], 
        contact_type = 'trans',
        view_df = regions,
        track = eigs[cond][['chrom', 'start', 'end', 'E1']],
        qrange = (0.02, 0.98),
        n_bins = 50,
        verbose = False
        )

In [ ]:
#Compartment strength - AA vs BB, in trans

In [ ]:
#AA vs BB compartment scores
def saddle_strengthAB(S, C):
    """
    Parameters
    ----------
    S, C : 2D arrays, square, same shape
        Saddle sums and counts, respectively
        
    Returns
    -------
    2x1D array
    Ratios of cumulative corner interaction scores, where the saddle data is 
    separately AA/AB and BB/BA corners with increasing extent
    
    """
    m, n = S.shape
    if m != n:
        raise ValueError("`saddledata` should be square.")

    ratiosA = np.zeros(n)
    for k in range(1, n):
        intra_sumA = S[n-k:n, n-k:n].sum() 
        intra_countA = C[n-k:n, n-k:n].sum()
        intraA = intra_sumA / intra_countA
        
        inter_sum = S[0:k, n-k:n].sum() + S[n-k:n, 0:k].sum()
        inter_count =  C[0:k, n-k:n].sum() + C[n-k:n, 0:k].sum()
        inter = inter_sum / inter_count
        
        ratiosA[k] = intraA / inter
        
    ratiosB = np.zeros(n)
    for k in range(1, n):
        intra_sumB = S[0:k, 0:k].sum()
        intra_countB = C[0:k, 0:k].sum()
        intraB = intra_sumB / intra_countB
        
        inter_sum = S[0:k, n-k:n].sum() + S[n-k:n, 0:k].sum()
        inter_count =  C[0:k, n-k:n].sum() + C[n-k:n, 0:k].sum()
        inter = inter_sum / inter_count
        
        ratiosB[k] = intraB / inter
    
    ratios = {
        'A' : ratiosA,
        'B' : ratiosB
    }
    
    return ratios

In [ ]:
#Own Eig1

In [ ]:
#AA vs BB strength, trans
strengthABTrans_ownE1 = {}
for cond in conditions:
    strengthABTrans_ownE1[cond] = saddle_strengthAB(trans_sums[cond], trans_counts[cond])

In [ ]:
#Plot bargraph of avg and dots of replicates - not vs control for AA and BB comp strength, trans
#5 bin square - top 10% A or B

comp_score_df_trans = pd.DataFrame(columns = ['Condition', 'Compartment', 'Replicate', 'Label', 'Score'])

for cond in conditions[0:8]:
    for comp in ['A', 'B']:
        comp_score_df_trans = comp_score_df_trans.append({
            'Condition' : cond, 
            'Compartment' : comp, 
            'Replicate' : repdict[cond], 
            'Label' : labeldict[cond],
            'Score' : strengthABTrans_ownE1[cond][comp][4]
        }, ignore_index = True)


In [ ]:
#https://stackoverflow.com/questions/64223870/seaborn-overlap-swarmplot-on-barplot
sns.set_style("ticks")
sns.set_context("paper")
cmap_bar = sns.color_palette(['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c'])
gs = GridSpec(nrows= 1, ncols=1, wspace = 0.6, hspace = 0.6)

plt.figure(figsize=(2, 2))

sns.stripplot(x='Compartment', y='Score', hue='Label', dodge=True, data=comp_score_df_trans, jitter = False, palette = cmap_bar)
ax1 = sns.barplot(x='Compartment', y='Score', hue='Label', data=comp_score_df_trans, palette = cmap_bar, alpha = 0.5, ci = False)
    
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[4:8], labels[4:8], bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
plt.ylim(0, 6)

plt.title(f'Interchromosomal\nCompartment Strength')
plt.ylabel('Saddle Strength')  
plt.xlabel('Compartment')
        
plt.savefig(f'{outDataDir}/figures/HiC_TransCompStrength_AllSamples_BarWithScatter_250kbbinEig1.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Density Plots and Scaling Plots
#https://cooltools.readthedocs.io/en/latest/notebooks/contacts_vs_distance.html

In [ ]:
#Calculate expected on 1kb binned coolers

In [ ]:
#coolers - 1kb bins
binsize = 1000

clr_paths_1kb = {}
for cond in conditions:
    clr_paths_1kb[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool::resolutions/{binsize}'

In [ ]:
for cond in conditions:
    in_fname = clr_paths_1kb[cond]
    region_fname = f'{outDataDir}/data/hg38_arms_filtered.bed'
    out_fname = f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.expected.cis.cli.tsv'
    !bsub -q short -W 01:00 -e ./lsf_jobs/LSB_%J.err -o ./lsf_jobs/LSB_%J.log \
        -n 8 -R span[hosts=1] -R select[ib] -R rusage[mem=2000] -R select[rh=8] -N \
        "cooltools expected-cis -p 8 -o $out_fname --ignore-diags 2 --view $region_fname $in_fname"

In [ ]:
#read in log binned expected for plotting
exp = {}
for cond in conditions:
    exp[cond] = pd.read_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.expected.cis.cli.tsv', sep = '\t')
 

In [ ]:
log_exp = {}
for cond in conditions:
    log_exp[cond] = cooltools.api.expected.logbin_expected(
        exp[cond], summary_name='balanced.sum', bins_per_order_magnitude=10, 
        bin_layout='fixed', min_nvalid=200, min_count=50)

In [ ]:
log_exp_agg = {}
for cond in conditions:
    log_exp_agg[cond] = cooltools.api.expected.combine_binned_expected(
        log_exp[cond][0], binned_exp_slope=log_exp[cond][1],
        Pc_name='balanced.avg', minmax_drop_bins=2, concat_original=False)

In [ ]:
#Add Density column - (balanced.avg * n_valid)/sum(balanced.avg * n_valid)
for cond in conditions:
    log_exp_agg[cond][0]['density'] = (
        log_exp_agg[cond][0]['balanced.avg'] * log_exp_agg[cond][0]['n_valid'])/np.nansum(
        log_exp_agg[cond][0]['balanced.avg'] * log_exp_agg[cond][0]['n_valid'])

In [ ]:
log_exp_agg[cond][0].head()

In [ ]:
#Smooth density before plotting, on log bins like for scaling. Then convert back to linear so can use semilogx
sns.set_style("ticks")
sns.set_context("paper")

from cooltools.lib import numutils
smoothf = der_smooth_function_combined=lambda x: numutils.robust_gauss_filter(x, 1.3)

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in ComboConds:
    ax.semilogx(
        10**smoothf(np.log10(log_exp_agg[cond][0]['dist.avg']*1000)),
        smoothf(log_exp_agg[cond][0]['density']),
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        ls = sampleLineStyles[cond]
    )

    ax.set(
        xlabel='separation, bp',
        ylabel='Density',
        xlim=(1e3,1e8),
        #ylim=(0, 0.07)
    )

    ax.grid(lw=0.5)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
    #ax.set_aspect(100)
    
plt.savefig(f'{outDataDir}/figures/R1R2_Density.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Plot scaling - combo with line at mitotic dropoff
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in ComboConds:
    ax.loglog(
        log_exp_agg[cond][0]['dist.avg']*1000,
        log_exp_agg[cond][0]['balanced.avg'],
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        ls = sampleLineStyles[cond],
        lw = 2
    )

ax.set(
    xlabel='separation (s), bp',
    ylabel='P(s)',
    xlim=(1e3,1e8))

ax.grid(lw=0.5)
ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
plt.title('P(s) Scaling Plot')

plt.axvline(x=1.5e7)
    
plt.savefig(f'{outDataDir}/figures/R1R2_Scaling_Combo_With1.5e7vline.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Plot scaling - combo 
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in ComboConds:
    ax.loglog(
        log_exp_agg[cond][0]['dist.avg']*1000,
        log_exp_agg[cond][0]['balanced.avg'],
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        ls = sampleLineStyles[cond],
        lw = 2
    )

ax.set(
    xlabel='separation (s), bp',
    ylabel='P(s)',
    xlim=(1e3,1e8))

ax.grid(lw=0.5)
ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
plt.title('P(s) Scaling Plot')
    
plt.savefig(f'{outDataDir}/figures/R1R2_Scaling_Combo.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Plot scaling and slope separately
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in SepConds:
    ax.loglog(
        log_exp_agg[cond][0]['dist.avg']*1000,
        log_exp_agg[cond][0]['balanced.avg'],
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        linestyle = sampleLineStyles[cond]
    )

    ax.set(
        xlabel='separation, bp',
        ylabel='IC contact frequency',
        xlim=(1e3,1e8)
    )
    #ax.set_aspect(1.0)
    ax.grid(lw=0.5)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
    
plt.savefig(f'{outDataDir}/figures/R1R2_Scaling_Separate.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Plot slope - combo
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in ComboConds:
    ax.semilogx(
        log_exp_agg[cond][1]['dist.avg']*1000,
        log_exp_agg[cond][1]['slope'],
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        ls = sampleLineStyles[cond],
        lw = 2
    )

ax.set(
    xlabel='separation (s), bp',
    ylabel='P(s) slope',
    ylim=(-2.5, 0),
    xlim=(1e3,1e8)
    )

ax.grid(lw=0.5)
ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
plt.title('P(s) Slope')
plt.axvline(x=1.5e7)
    
plt.savefig(f'{outDataDir}/figures/ScalingSlope_Combo_With1.5e7vline.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Plot slope - combo
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in ComboConds:
    ax.semilogx(
        log_exp_agg[cond][1]['dist.avg']*1000,
        log_exp_agg[cond][1]['slope'],
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        ls = sampleLineStyles[cond],
        lw = 2
    )

ax.set(
    xlabel='separation (s), bp',
    ylabel='P(s) slope',
    ylim=(-2.5, 0),
    xlim=(1e3,1e8)
    )

ax.grid(lw=0.5)
ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)

plt.title('P(s) Slope')
    
plt.savefig(f'{outDataDir}/figures/ScalingSlope_Combo.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Plot scaling and slope separately
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(
    figsize=(2, 2))

for cond in SepConds:
    ax.semilogx(
        log_exp_agg[cond][1]['dist.avg']*1000,
        log_exp_agg[cond][1]['slope'],
        color = sampleColors[cond],
        label = samplePlotNames[cond],
        linestyle = sampleLineStyles[cond]
    )

    ax.set(
        xlabel='separation, bp',
        ylabel='slope',
        ylim=(-2.2, 0),
        xlim=(1e3,1e8)
    )

    ax.grid(lw=0.5)
    #ax.set_aspect(1)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left", frameon = False)
    
plt.savefig(f'{outDataDir}/figures/R1R2_ScalingSlope_Separate.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#Call insulation, pileup at control boundaries

In [ ]:
#coolers - 10kb bins
binsize = 10000

clr_paths_10kb = {}
for cond in conditions:
    clr_paths_10kb[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
    
clrs10kb = {
    cond: cooler.Cooler(clr_paths_10kb[cond]) for cond in conditions
}

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
# create a view with chromosome arms using chromosome sizes and definition of centromeres
hg38_arms = bioframe.make_chromarms(hg38_chromsizes,  hg38_cens)

In [ ]:
# Select only chromosomes that are present in the good chromosomes
hg38_chromsizes = hg38_chromsizes.loc[good_chroms]
hg38_arms = hg38_arms[hg38_arms.chrom.isin(good_chroms)].reset_index(drop=True)
hg38_chrom_vf = bioframe.make_viewframe(hg38_chromsizes)

In [ ]:
#call insulation
insulation_table = {}

for cond in conditions:
    insulation_table[cond] = insulation(clrs10kb[cond], 250000, verbose=False, ignore_diags = 2, view_df=hg38_chrom_vf)

In [ ]:
insulation_table[cond]

In [ ]:
# Functions to help with plotting
def pcolormesh_45deg(ax, matrix_c, start=0, resolution=1, *args, **kwargs):
    start_pos_vector = [start+resolution*i for i in range(len(matrix_c)+1)]
    import itertools
    n = matrix_c.shape[0]
    t = np.array([[1, 0.5], [-1, 0.5]])
    matrix_a = np.dot(np.array([(i[1], i[0])
                                for i in itertools.product(start_pos_vector[::-1],
                                                           start_pos_vector)]), t)
    x = matrix_a[:, 1].reshape(n + 1, n + 1)
    y = matrix_a[:, 0].reshape(n + 1, n + 1)
    im = ax.pcolormesh(x, y, np.flipud(matrix_c), *args, **kwargs)
    im.set_rasterized(True)
    return im

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

In [ ]:
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid import make_axes_locatable
import bioframe

sns.set_style("ticks")
sns.set_context("paper")

fig = plt.figure(figsize=(8, 5))

gs0 = GridSpec(nrows = 5, ncols = 2, width_ratios = [50, 1], hspace = 0.5)

for i, cond in enumerate(ComboConds):
    ax = plt.subplot(gs0[i, 0])
    region = ('chr14', 70000000, 80000000)
    norm = LogNorm(vmax=0.1, vmin=0.001)
    data = clrs10kb[cond].matrix(balance=True).fetch(region)
    im = pcolormesh_45deg(ax, data, start=region[1], resolution=10000, norm=norm, cmap='fall')
    ax.set_ylim(0, 2000000)
    format_ticks(ax, rotate=False)
    ax.xaxis.set_visible(False)
    ax.set_xlim(region[1], region[2])
    plt.title(samplePlotNames[cond])
    plt.ylabel('Distance')

    #divider = make_axes_locatable(ax)
    cax = plt.subplot(gs0[i, 1])
    plt.colorbar(im, cax=cax)
    cax.set_aspect(6)

#ins_ax = divider.append_axes("bottom", size="50%", pad=0., sharex=ax)
    
for cond in ComboConds:
    ins_ax = plt.subplot(gs0[4, 0])
    insul_region = bioframe.select(insulation_table[cond], region)
    ins_ax.plot(insul_region[['start', 'end']].mean(axis=1), 
                insul_region['log2_insulation_score_250000'], 
                label=samplePlotNames[cond], color = sampleColors[cond])
    plt.ylim(-2.25, 1.25)
    format_ticks(ins_ax, y=False, rotate=False)
    ins_ax.legend().set_visible(False)
    ins_ax.set_xlim(region[1], region[2])
    plt.ylabel('Insulation')
    plt.xlabel('Chr14 Coordinates')
    
    h, l = ins_ax.get_legend_handles_labels() # get labels and handles
    leg_ax = plt.subplot(gs0[4, 1])
    leg_ax.legend(h, l, frameon = False, bbox_to_anchor=(0, 1), loc="upper left")  
    leg_ax.axis('off')
    
plt.savefig(f'{outDataDir}/figures/All_insulation_example.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
from skimage.filters import threshold_li, threshold_otsu
windows = [250000]
histkwargs = dict(
    bins=10**np.linspace(-4,1,200),
    histtype='step',
    lw=2,
)

fig = plt.figure(figsize=(12, 16))

gs0 = GridSpec(nrows = 4, ncols = 2, wspace=0.3, hspace = 0.3)

thresholds_li = {}
thresholds_otsu = {}
bounds_li = {}
bounds_otsu = {}

for i, cond in enumerate(ComboConds):
    gs1 = GridSpecFromSubplotSpec(nrows=len(windows), ncols=1, wspace = 0.3, hspace = 1, subplot_spec = gs0[i])

    thresholds_li[cond] = {}
    thresholds_otsu[cond] = {}
    bounds_li[cond] = {}
    bounds_otsu[cond] = {}

    for i, w in enumerate(windows):
        ax = plt.subplot(gs1[i])
        ax.hist(
            insulation_table[cond][f'boundary_strength_{w}'],
            **histkwargs
        )
        thresholds_li[cond][w] = threshold_li(insulation_table[cond][f'boundary_strength_{w}'].dropna().values)
        thresholds_otsu[cond][w] = threshold_otsu(insulation_table[cond][f'boundary_strength_{w}'].dropna().values)
        n_boundaries_li = (insulation_table[cond][f'boundary_strength_{w}'].dropna()>=thresholds_li[cond][w]).sum()
        bounds_li[cond][w] = insulation_table[cond][(insulation_table[cond][f'boundary_strength_{w}']>=thresholds_li[cond][w])]
        n_boundaries_otsu = (insulation_table[cond][f'boundary_strength_{w}'].dropna()>=thresholds_otsu[cond][w]).sum()
        bounds_otsu[cond][w] = insulation_table[cond][(insulation_table[cond][f'boundary_strength_{w}']>=thresholds_otsu[cond][w])]

        ax.axvline(thresholds_li[cond][w], c='green')
        ax.axvline(thresholds_otsu[cond][w], c='magenta')
        ax.text(0.01, 0.9,
                 f'Window {w//1000}kb',
                 ha='left',
                 va='top',
                 transform=ax.transAxes)
        ax.text(0.01, 0.7,
                f'{n_boundaries_otsu} boundaries (Otsu)',
                c='magenta',
                ha='left',
                va='top',
                transform=ax.transAxes)
        ax.text(0.01, 0.5,
                f'{n_boundaries_li} boundaries (Li)',
                c='green',
                ha='left',
                va='top',
                transform=ax.transAxes)

        ax.set(
            xscale='log',
            ylabel='# boundaries'
        )

        ax.set(xlabel='Boundary strength')
        plt.title(cond)
    
plt.savefig(f'{outDataDir}/figures/AllCond_BoundaryThresholds.png', dpi = 300, 
            bbox_inches = 'tight')

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')

In [ ]:
hg38_chromsizes = hg38_chromsizes.loc[good_chroms]

In [ ]:
for cond in conditions:
    for w in windows:
        bioframe.to_bigwig(insulation_table[cond], hg38_chromsizes,
            f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.insul_score_{w}.bw', 
            f'log2_insulation_score_{w}')

In [ ]:
import bbi

In [ ]:
# Create stackup. flank = .5Mb, nbins = 100, 250kb window, 10kb hi-c resolution
flank = 500000 # Length of flank to one side from the boundary, in basepairs
nbins = 100   # Number of bins to split the region
ins_pileup_signal = {}
for cond in conditions:
    ins_pileup_signal[cond] = bbi.stackup(
        f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.insul_score_250000.bw',
        bounds_otsu['WT_Ctrl_R1R2'][250000].chrom,
        bounds_otsu['WT_Ctrl_R1R2'][250000].start-flank,
        bounds_otsu['WT_Ctrl_R1R2'][250000].end+flank, bins=nbins)

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")

f, ax = plt.subplots(figsize=[2, 2])
flank = 500000
nbins = 100
x = np.linspace(-flank/1e3, flank/1e3, nbins)
for cond in SepConds:
    ax.plot(x, np.nanmean(ins_pileup_signal[cond], axis=0), color = sampleColors[cond], label = cond, 
            ls = sampleLineStyles[cond], lw = 2)
plt.xlim(-flank/1e3, flank/1e3)
plt.ylim(-1, .3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.1, frameon = False)
plt.title('Insulation Profile \nWT Ctrl (R1+R2) Boundaries')
plt.ylabel('Average Insulation')
plt.xlabel('Distance from Boundary (Kb)')
plt.savefig(f'{outDataDir}/figures/InsulationPileup_WTCtrlR1R2Boundaries.png', dpi = 300, 
            bbox_inches = 'tight')

In [ ]:
#separate out TAD vs Compartment boundaries

In [ ]:
#called boundaries from cooler compartments (with gene density)
binsize_comp = 250000

eigs = {}
for cond in conditions:
    eigs[cond] = pd.read_csv(f'{outDataDir}/data/{long_names[cond]}.{binsize_comp//1000}kb.mapq30.byarm.eigs.cis.vecs.txt', sep='\t')

#calling A vs B compartments in each sample
eigsA = {}
for cond in conditions:
    eigsA[cond] = eigs[cond][eigs[cond]['E1'] > 0]
    
eigsB = {}
for cond in conditions:
    eigsB[cond] = eigs[cond][eigs[cond]['E1'] < 0]

compRangesA = {}
for cond in conditions:
    compRangesA[cond] = bioframe.merge(pd.DataFrame(data = {
        'chrom' : eigsA[cond].chrom,
        'start' : eigsA[cond].start,
        'end' : eigsA[cond].end,
        'name' : '.',
        'score' : '.',
        'strand' : '+'
    }))
    
compRangesB = {}
for cond in conditions:
    compRangesB[cond] = bioframe.merge(pd.DataFrame(data = {
        'chrom' : eigsB[cond].chrom,
        'start' : eigsB[cond].start,
        'end' : eigsB[cond].end,
        'name' : '.',
        'score' : '.',
        'strand' : '+'
    }))

In [ ]:
#calling compartment boundaries in each sample
#Calculate which bins flank sign changes - on same chromosome. 

compBounds = {}
signsAll = {}

for cond in conditions:    
    signsAll[cond] = pd.DataFrame(data = {
        'Chrom1' : eigs[cond]['chrom'][0:-1].reset_index(drop = True),
        'Bin1Start' : eigs[cond]['start'][0:-1].reset_index(drop = True),
        'Bin1End' : eigs[cond]['end'][0:-1].reset_index(drop = True),
        'Bin1E1' : eigs[cond]['E1'][0:-1].reset_index(drop = True),
        'Chrom2' : eigs[cond]['chrom'][1:].reset_index(drop = True),
        'Bin2Start' : eigs[cond]['start'][1:].reset_index(drop = True),
        'Bin2End' : eigs[cond]['end'][1:].reset_index(drop = True),  
        'Bin2E1' : eigs[cond]['E1'][1:].reset_index(drop = True),
        'sign' : (eigs[cond]['E1'][0:-1].reset_index(drop = True) * eigs[cond]['E1'][1:].reset_index(drop = True))
        })
    signsAll[cond].apply(pd.to_numeric, errors='ignore')
    signsAll[cond] = signsAll[cond].astype({"Bin1Start": int, "Bin1End": int, "Bin2Start": int, "Bin2End": int})

    compBounds[cond] = signsAll[cond][(signsAll[cond]['sign'] < 0) &
                                      (signsAll[cond]['Chrom1'] == signsAll[cond]['Chrom2']) 
                                     ]

In [ ]:
compBoundsRanges = {}
for cond in conditions:
    compBoundsRanges[cond] = pd.DataFrame(data = {
        'chrom' : compBounds[cond].Chrom1,
        'start' : compBounds[cond].Bin1Start,
        'end' : compBounds[cond].Bin2End,
        'name' : '.',
        'score' : compBounds[cond].sign,
        'strand' : '+'
    })

In [ ]:
compBoundsRanges['WT_Ctrl_R1R2']

In [ ]:
bounds_otsu[cond][250000]

In [ ]:
tadOnlyBoundaries = {}
overlapCompBoundaries = {}

for cond in ComboConds:
    insBoundaries = bounds_otsu[cond][250000].copy()
    compBoundaries = compBoundsRanges[cond].copy()
    overlapBounds = bioframe.overlap(insBoundaries, compBoundaries, how = 'inner').iloc[:, 0:4]
    overlapBounds.columns = ['chrom', 'start', 'end', 'boundary_strength_250000']
    overlapCompBoundaries[cond] = overlapBounds.drop_duplicates()
    tadOnlyBoundaries[cond] = bioframe.subtract(insBoundaries, overlapCompBoundaries[cond]).drop_duplicates()

In [ ]:
#save the comp vs tad insulation boundaries, and all insulation boundaries
for cond in ComboConds:
    overlapCompBoundaries[cond].to_csv(
        f'{outDataDir}/data/{long_names[cond]}.compartmentOverlap_insulation_250kbWindow_bounds.mapq30.goodchroms.arms.txt',
        sep = "\t", index = False)
    tadOnlyBoundaries[cond].to_csv(    
        f'{outDataDir}/data/{long_names[cond]}.TADonly_insulation_250kbWindow_bounds.mapq30.goodchroms.arms.txt',
        sep = "\t", index = False)
    bounds_otsu[cond][250000].to_csv(    
        f'{outDataDir}/data/{long_names[cond]}.AllBounds_insulation_250kbWindow_bounds.mapq30.goodchroms.arms.txt',
        sep = "\t", index = False)

In [ ]:
#read in comp vs tad insulation boundaries
tadOnlyBoundaries = {}
overlapCompBoundaries = {}

for cond in ComboConds:
    overlapCompBoundaries[cond] = pd.read_csv(
        f'{outDataDir}/data/{long_names[cond]}.compartmentOverlap_insulation_250kbWindow_bounds.mapq30.goodchroms.arms.txt',
        sep = "\t")
    tadOnlyBoundaries[cond] = pd.read_csv(    
        f'{outDataDir}/data/{long_names[cond]}.TADonly_insulation_250kbWindow_bounds.mapq30.goodchroms.arms.txt',
        sep = "\t")
    

In [ ]:
# Create stackup. flank = .5Mb, nbins = 100, 250kb window, 10kb hi-c resolution, TAD only boundaries
flank = 500000 # Length of flank to one side from the boundary, in basepairs
nbins = 100   # Number of bins to split the region
ins_pileup_signal_TAD = {}
for cond in conditions:
    ins_pileup_signal_TAD[cond] = bbi.stackup(
        f'{outDataDir}/data/{long_names[cond]}.{binsize//1000}kb.mapq30.insul_score_250000.bw',
        tadOnlyBoundaries['WT_Ctrl_R1R2'].chrom,
        tadOnlyBoundaries['WT_Ctrl_R1R2'].start-flank,
        tadOnlyBoundaries['WT_Ctrl_R1R2'].end+flank, bins=nbins)

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")
f, ax = plt.subplots(figsize=[2, 2])
flank = 500000
nbins = 100
x = np.linspace(-flank/1e3, flank/1e3, nbins)
for cond in SepConds:
    ax.plot(x, np.nanmean(ins_pileup_signal_TAD[cond], axis=0), color = sampleColors[cond], label = samplePlotNames[cond], 
            ls = sampleLineStyles[cond], lw = 2)
plt.xlim(-flank/1e3, flank/1e3)
plt.ylim(-.8, .3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.1, frameon = False)
plt.title('Insulation Profile \n WT Ctrl Boundaries')
plt.ylabel('Average Insulation')
plt.xlabel('Distance from Boundary (Kb)')
plt.savefig(f'{outDataDir}/figures/InsulationPileup_WTCtrlR1R2TADBoundaries.png', dpi = 300, 
            bbox_inches = 'tight')

In [ ]:
sns.set_style("ticks")
sns.set_context("paper")
f, ax = plt.subplots(figsize=[2, 2])
flank = 500000
nbins = 100
x = np.linspace(-flank/1e3, flank/1e3, nbins)
for cond in ComboConds:
    ax.plot(x, np.nanmean(ins_pileup_signal_TAD[cond], axis=0), color = sampleColors[cond], label = samplePlotNames[cond], 
            ls = sampleLineStyles[cond], lw = 2)
plt.xlim(-flank/1e3, flank/1e3)
plt.ylim(-.8, .3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.1, frameon = False)
plt.title('Insulation Profile \n WT Ctrl TAD Boundaries')
plt.ylabel('Average Insulation')
plt.xlabel('Distance from Boundary (Kb)')
plt.savefig(f'{outDataDir}/figures/InsulationPileup_WTCtrlR1R2TADBoundaries_Combo.png', dpi = 300, 
            bbox_inches = 'tight')

In [ ]:
#1kb bins - coverage in cis vs total

In [ ]:
#coolers - 1kb bins
binsize = 1000

clr_paths_1kb = {}
for cond in conditions:
    clr_paths_1kb[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
    
clrs1kb = {
    cond: cooler.Cooler(clr_paths_1kb[cond]) for cond in conditions
}

In [ ]:
clrs1kb['WT_Ctrl_R1'].info

In [ ]:
ciscov = {}
totcov = {}
for cond in conditions:
    ciscov[cond], totcov[cond] = cooltools.api.coverage.coverage(
        clrs1kb[cond], 
        ignore_diags=2, 
        chunksize=10000000)

In [ ]:
#total sum is not the same as sum/count from cooler unless don't ignore 2 diags, but then cis % is not right (since not balanced)

In [ ]:
ciscov[cond]

In [ ]:
#cis/trans - all chromosomes, ignore first 2 diags, from coolers
cisratio = {}
for cond in conditions:
    cisratio[cond] = sum(ciscov[cond])/(sum(totcov[cond])-sum(ciscov[cond]))

In [ ]:
#Plot bargraph of avg and dots of replicates for cis ratio

cis_ratio_df = pd.DataFrame(columns = ['Condition', 'Replicate', 'Label', 'Ratio'])


In [ ]:
for cond in SepConds:
    cis_ratio_df = cis_ratio_df.append({
                'Condition' : f'{cond}', 
                'Replicate' : repdict[cond], 
                'Label' : f'{labeldict[cond]}',
                'Ratio' : cisratio[cond]
            }, ignore_index = True)
    

In [ ]:
cis_ratio_df

In [ ]:
#https://stackoverflow.com/questions/64223870/seaborn-overlap-swarmplot-on-barplot
sns.set_style("ticks")
sns.set_context("paper")
cmap_bar = sns.color_palette(['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c'])
gs = GridSpec(nrows= 1, ncols=1, wspace = 0.6, hspace = 0.6)

plt.figure(figsize=(2, 2))

ax = plt.subplot(gs[0])

sns.stripplot(
    x='Label',
    y='Ratio',
    hue='Label', 
    dodge=False, 
    data=cis_ratio_df, 
    jitter = False, 
    palette = cmap_bar, 
    ax = ax)
ax1 = sns.barplot(
    x='Label', 
    y='Ratio', 
    hue='Label', 
    data=cis_ratio_df,
    palette = cmap_bar, 
    alpha = 0.5, 
    dodge = False,
    ci = False, 
    ax = ax)
    
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[4:8], labels[4:8], bbox_to_anchor=(1.04,1), frameon = False)
 
plt.title(f'Cis/Trans Ratio')
plt.ylabel('Cis/Trans Contacts')  
#plt.ylim(0, 1)
plt.xlabel('Sample')
plt.xticks(rotation=90)
        
plt.savefig(f'{outDataDir}/figures/R1R2_HiC_CisvsTransRatio_BarWithScatter.png', dpi = 300, bbox_inches = "tight")

In [ ]:
#cis/trans - all chromosomes, ignore first 2 diags, from coolers
cisratio = {}
for cond in conditions:
    cisratio[cond] = sum(ciscov[cond])/sum(totcov[cond])

In [ ]:
#Plot bargraph of avg and dots of replicates for cis ratio

cis_ratio_df = pd.DataFrame(columns = ['Condition', 'Replicate', 'Label', 'Ratio'])


In [ ]:
for cond in SepConds:
    cis_ratio_df = cis_ratio_df.append({
                'Condition' : f'{cond}', 
                'Replicate' : repdict[cond], 
                'Label' : f'{labeldict[cond]}',
                'Ratio' : cisratio[cond]
            }, ignore_index = True)
    

In [ ]:
cis_ratio_df

In [ ]:
#https://stackoverflow.com/questions/64223870/seaborn-overlap-swarmplot-on-barplot
sns.set_style("ticks")
sns.set_context("paper")
cmap_bar = sns.color_palette(['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c'])
gs = GridSpec(nrows= 1, ncols=1, wspace = 0.6, hspace = 0.6)

plt.figure(figsize=(2, 2))

ax = plt.subplot(gs[0])

sns.stripplot(
    x='Label',
    y='Ratio',
    hue='Label', 
    dodge=False, 
    data=cis_ratio_df, 
    jitter = False, 
    palette = cmap_bar, 
    ax = ax)
ax1 = sns.barplot(
    x='Label', 
    y='Ratio', 
    hue='Label', 
    data=cis_ratio_df,
    palette = cmap_bar, 
    alpha = 0.5, 
    dodge = False,
    ci = False, 
    ax = ax)
    
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[4:8], labels[4:8], bbox_to_anchor=(1.04,1), frameon = False)
 
plt.title(f'Fraction Intrachromosomal')
plt.ylabel('Cis/Total Contacts')  
plt.ylim(0.8, 0.9)
plt.xlabel('Sample')
plt.xticks(rotation=90)
        
plt.savefig(f'{outDataDir}/figures/R1R2_HiC_CisvsTotalRatio_BarWithScatter.png', dpi = 300, bbox_inches = "tight")